In [54]:
import sys

from pprint import pprint
from typing import Dict, List, Tuple, Optional
import os
import numpy as np
import timeit
import statistics
import matplotlib.pyplot as plt
import re
from scipy.special import factorial

In [2]:
def read_from_file_1(file: str = "data1.txt"):
    D = []
    f = open(file, "r")
    for row in f:
        data = row.strip().split(' ')
        D += data
    f.close()
    return np.array(D).astype(np.int64)

In [3]:
def count_increases(D) -> int:
    increases = 0
    for i in range(0,D.shape[0]-1):
        if D[i]<D[i+1]:
            increases += 1
    return increases

In [4]:
def count_increases_sliding_window(D) -> int:
    increases = 0
    for i in range(0,D.shape[0]-3):
        if (D[i]+D[i+1]+D[i+2])<(D[i+1]+D[i+2]+D[i+3]):
            increases += 1
    return increases

In [5]:
def solve_1():
    D = read_from_file_1()
    print("The solution for 1.1 is:",count_increases(D))
    print("The solution for 1.2 is:",count_increases_sliding_window(D))

In [6]:
def location(file: str = "data2.txt"):
    D = []
    f = open(file, "r")
    hor = 0
    depth = 0
    for row in f:
        data = row.strip().split(' ')
        pos_change = int(data[1])
        if data[0] == "forward":
            hor += pos_change
        elif data[0] == "down":
            depth += pos_change
        else:
            depth -= pos_change
            
    
    return hor, depth

In [7]:
def location_with_aim(file: str = "data2.txt"):
    D = []
    f = open(file, "r")
    hor = 0
    depth = 0
    aim = 0
    for row in f:
        data = row.strip().split(' ')
        pos_change = int(data[1])
        if data[0] == "forward":
            hor += pos_change
            depth += pos_change * aim
        elif data[0] == "down":
            aim += pos_change
        else:
            aim -= pos_change
            
    return hor, depth, aim

In [8]:
def solve_2():
    hor, depth = location("data2.txt")
    mult1 = hor*depth
    print("The solution for 2.1 is:",mult1)
    hor, depth, aim = location_with_aim("data2.txt")
    mult2 = hor*depth
    print("The solution for 2.2 is:",mult2)

In [9]:
def read_from_file_3(file: str = "data3.txt"):
    D = []
    f = open(file, "r")
    for row in f:
        data = row.strip().split()
        data[0] = list(data[0])
        D += data
    f.close()
    return np.array(D).astype(np.int64)

In [10]:
def convert_to_bin(A):
    lenght = len(A)
    return [A[i] * (2**(lenght-i-1)) for i in range(0,lenght)]

In [11]:
def decode(D):
    lenght = D.shape[1]
    gamma_rate = np.zeros(lenght)
    epsilon_rate = np.zeros(lenght)
    for i in range(0, lenght):
        if np.mean(D,axis=0)[i] > 0.5:
            gamma_rate[i] = 1  
        else:
            epsilon_rate[i] = 1
    gamma_rate = convert_to_bin(gamma_rate)
    epsilon_rate = convert_to_bin(epsilon_rate)
    gamma_epsilon = np.sum(gamma_rate)*np.sum(epsilon_rate)
    return gamma_epsilon
            

In [12]:
def oxygen(D):
    i = 0
    while D.shape[0] > 1:
        if np.mean(D,axis=0)[i] >= 0.5:
            indexes = np.argwhere(D[:,i]==1).flatten()
        else:
            indexes = np.argwhere(D[:,i]==0).flatten()
        D = D[indexes,:]
        i += 1
    oxygen = np.sum(convert_to_bin(D.flatten()))
    return oxygen

In [13]:
def co2(D):
    i = 0
    while D.shape[0] > 1:
        if np.mean(D,axis=0)[i] < 0.5:
            indexes = np.argwhere(D[:,i]==1).flatten()
        else:
            indexes = np.argwhere(D[:,i]==0).flatten()
        D = D[indexes,:]
        i += 1
    co2 = np.sum(convert_to_bin(D.flatten()))
    return co2

In [14]:
def solve_3():
    D = read_from_file_3("data3.txt")
    gamma_epsilon = int(decode(D))
    mult = oxygen(D)*co2(D)
    print("The solution for 3.1 is:",gamma_epsilon)
    print("The solution for 3.2 is:",mult)

In [15]:
def read_from_file_4(file: str = "data4.txt"):
    draws = []
    boards = []
    f = open(file, "r")
    draws = f.readline().strip().split(',')
    board = np.zeros(shape=(5,5))
    i = 0
    for row in f:
        if i == 5:
            i = 0
            boards.append(board.astype(np.int64))
        if row == '\n':
            board = np.zeros(shape=(5,5))
            continue
        board[i,:] = row.strip().split()
        i += 1
    boards.append(board.astype(np.int64))
    f.close()
    return np.array(draws).astype(np.int64), boards

In [16]:
def findwinners(boards):
    counter = len(boards)
    winners = []
    for i in range(0,counter):
        for row in boards[i]:
            if np.all(row==-1) and i not in winners:
                winners.append(i)
                continue
        for column in boards[i].T:
            if np.all(column==-1) and i not in winners:
                winners.append(i)
                continue
    return winners

In [17]:
def play_to_win(file: str = "data4.txt"):
    draws, boards = read_from_file_4(file)
    index_of_winning_board = -1
    for draw in draws:
        for board in boards:
            pos = np.where(board == draw)
            if pos != []:
                board[pos] = -1
        indexes_of_winning_boards = findwinners(boards)
        if indexes_of_winning_boards != []:
            winning_board = boards[indexes_of_winning_boards[0]]
            winning_draw = draw
            break
    
    return winning_board, winning_draw

In [18]:
def calc_score(board, winning_draw):
    indexes = np.where(board == -1)
    board[indexes] = 0
    return np.sum(board) * winning_draw

In [19]:
def play_to_lose(file: str = "data4.txt"):
    draws, boards = read_from_file_4(file)
    for draw in draws:
        for board in boards:
            pos = np.where(board == draw)
            if pos != []:
                board[pos] = -1
        indexes_of_winning_boards = findwinners(boards)
        if len(boards) == 1 and indexes_of_winning_boards != []:
            return boards[0], draw
        if indexes_of_winning_boards != []:
            for i in reversed(indexes_of_winning_boards):
                boards.pop(i)      
        
    return -1, -1

In [20]:
def solve_4():
    winning_board, winning_draw = play_to_win("data4.txt")
    losing_board, losing_draw = play_to_lose("data4.txt")
    winning_score = calc_score(winning_board, winning_draw)
    losing_score = calc_score(losing_board, losing_draw)
    print("The solution for 4.1 is:",winning_score)
    print("The solution for 4.2 is:",losing_score)

In [21]:
class Coordinate:
    def __init__(self, x_1,y_1,x_2,y_2)->None:
        self.x1 = x_1
        self.y1 = y_1
        self.x2 = x_2
        self.y2 = y_2
    
    def get_all_points_on_lines(self):
        points = []
        if self.x1 != self.x2 and self.y1 != self.y2:
            if self.x1<self.x2:
                if self.y1<self.y2:
                    p = []
                    q = []
                    leng = self.x2+1 - self.x1
                    for i in range(0, leng):
                        p.append(self.x1+i)
                        q.append(self.y1+i)
                    points = [p,q]
                else:
                    p = []
                    q = []
                    leng = self.x2+1 - self.x1
                    for i in range(0, leng):
                        p.append(self.x1+i)
                        q.append(self.y1-i)
                    points = [p,q]
            elif self.x1>self.x2:
                if self.y1<self.y2:
                    p = []
                    q = []
                    leng = self.x1+1 - self.x2
                    for i in range(0, leng):
                        p.append(self.x1-i)
                        q.append(self.y1+i)
                    points = [p,q]
                else:
                    p = []
                    q = []
                    leng = self.x1+1 - self.x2
                    for i in range(0, leng):
                        p.append(self.x1-i)
                        q.append(self.y1-i)
                    points = [p,q]
                
        else:   
            if self.y1 < self.y2:
                p = [self.x1]
                q = []
                for i in range(self.y1, self.y2+1):
                    q.append(i)
                points = [p,q]
            elif self.y1 > self.y2:
                p = [self.x1]
                q = []
                for i in range(self.y2, self.y1+1):
                    q.append(i)
                points = [p,q]
            elif self.x1 < self.x2:
                p = []
                q = [self.y1]
                for i in range(self.x1, self.x2+1):
                    p.append(i)
                points = [p,q]
            else:
                p = []
                q = [self.y1]
                for i in range(self.x2, self.x1+1):
                    p.append(i)
                points = [p,q]
        return points

In [22]:
def read_from_file_5(file: str = "data5.txt"):
    coordinates = []
    largest_x = 0
    largest_y = 0
    f = open(file, "r")
    for row in f:
        data = re.split(',|->',row.strip())
        co = Coordinate(int(data[0]),int(data[1]),int(data[2]),int(data[3]))
        if int(data[0])>largest_x: largest_x = int(data[0])
        if int(data[2])>largest_x: largest_x = int(data[2]) 
        if int(data[1])>largest_y: largest_y = int(data[1]) 
        if int(data[3])>largest_y: largest_y = int(data[3])
        coordinates.append(co)
        
    f.close()
    return coordinates, largest_x, largest_y

In [23]:
def get_diagram(coordinates, largest_x, largest_y):
    diagram = np.zeros(shape=(largest_x+1, largest_y+1))
    for curr in coordinates:
        if curr.x1 != curr.x2 and curr.y1 != curr.y2:
            continue
        diagram[curr.get_all_points_on_lines()] += 1
    return diagram

In [24]:
def get_danger(diagram):
    indexes = np.where(diagram < 2)
    diagram[indexes] = 0
    indexes = np.where(diagram > 1)
    diagram[indexes] = 1
    return int(np.sum(diagram))

In [25]:
def get_diagram_with_diag(coordinates, largest_x, largest_y):
    diagram = np.zeros(shape=(largest_x+1, largest_y+1))
    for curr in coordinates:
        #print(curr.get_all_points_on_lines())
        diagram[curr.get_all_points_on_lines()] += 1
    return diagram

In [26]:
def solve_5():
    coordinates, largest_x, largest_y = read_from_file_5("data5.txt")
    di = get_diagram(coordinates, largest_x, largest_y)
    print("The solution for 5.1 is:",get_danger(di))
    di_diag = get_diagram_with_diag(coordinates, largest_x, largest_y)
    print("The solution for 5.2 is:",get_danger(di_diag))

In [27]:
def read_from_file_6(file: str = "data6.txt"):
    fish = []
    f = open(file, "r")
    for row in f:
        fish = row.strip().split(',')
        fish = [int(f1) for f1 in fish]
    f.close()
    return fish

In [28]:
def sim_lanternfish(num_of_days, fish):
    x = 0
    for i in range(0,num_of_days):
        new = 0
        for f in range(0,len(fish)):
            if fish[f] == 0:
                new += 1
                fish[f] = 6
            else:
                fish[f] -=1
        for i in range(0,new):
            fish.append(8)
    return fish

In [29]:
def write_array(fish):
    numpy_fish = np.zeros(shape=(9))
    for f in fish:
        numpy_fish[f] += 1
    return numpy_fish

In [30]:
def sim_lanternfish_well(num_of_days, fish_long):
    fish = write_array(fish_long)
    for i in range(0,num_of_days):
        changing_fish = fish[0]
        fish[0:6] = fish[1:7]
        fish[6] = changing_fish + fish[7]
        fish[7] = fish[8]
        fish[8] = changing_fish
    return fish
        

In [36]:
def solve_6():
    fish1 = read_from_file_6("data6.txt")
    print("The solution for 6.1 is:",len(sim_lanternfish(80,fish1)))
    fish2 = read_from_file_6("data6.txt")
    x = sim_lanternfish_well(256, fish2)
    print("The solution for 6.2 is:",int(np.sum(x)))

In [51]:
def read_from_file_7(file: str = "data7.txt"):
    crabs = np.array(())
    f = open(file, "r")
    for row in f:
        crabs_ = row.strip().split(',')
        crabs = np.array(crabs_).astype(np.int64)
    f.close()
    return crabs

In [58]:
def find_alignment(crabs):
    lowest = np.min(crabs)
    highest = np.max(crabs)
    lowest_cost = (np.Inf,np.Inf)
    for i in range(lowest,highest+1):
        cost = np.sum(abs(crabs - i))
        if cost<lowest_cost[0]:
            lowest_cost = (int(cost),i)
    return lowest_cost

In [59]:
def find_alignment_expensive(crabs):
    lowest = np.min(crabs)
    highest = np.max(crabs)
    lowest_cost = (np.Inf,np.Inf)
    for i in range(lowest,highest+1):
        n= abs(crabs - i)
        cost = np.sum(n*(n+1)/2)
        if cost<lowest_cost[0]:
            lowest_cost = (int(cost),i)
    return lowest_cost

In [63]:
def solve_7():
    crabs = read_from_file_7("data7.txt")
    al1 = find_alignment(crabs)
    al2 = find_alignment_expensive(crabs)
    print("The solution for 7.1 is:",al1[0])
    print("The solution for 7.2 is:",al2[0])

In [64]:
solve_1()
solve_2()
solve_3()
solve_4()
solve_5()
solve_6()
solve_7()

The solution for 1.1 is: 1374
The solution for 1.2 is: 1418
The solution for 2.1 is: 1654760
The solution for 2.2 is: 1956047400
The solution for 3.1 is: 852500
The solution for 3.2 is: 1007985
The solution for 4.1 is: 12796
The solution for 4.2 is: 18063
The solution for 5.1 is: 7674


C:\Users\CK\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  
C:\Users\CK\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  """


The solution for 5.2 is: 20898
The solution for 6.1 is: 386536
The solution for 6.2 is: 1732821262171
The solution for 7.1 is: 329389
The solution for 7.2 is: 86397080
